In [2]:
# Import Dependencies
import pandas as pd
import numpy as np
import json
import ast
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine



In [7]:
# input file names
keywords_csv = "keywords.csv"
keywords_df = pd.read_csv(keywords_csv,low_memory=False)
movies_metadata_csv = "movies_metadata.csv"

In [8]:
def load_keywords(file_path):
    df = pd.read_csv(file_path, dtype='unicode')
    
    # json columns
    json_columns = ['keywords']
    for column in json_columns:
       
        # use ast because json data has single quotes in the csv, which is invalid for a json object; it should be " normally
        df[column] = df[column].apply(lambda x: np.nan if pd.isnull(x) else ast.literal_eval(x))
    return df


In [9]:
def load_movies_metadata(file_path):
    df = pd.read_csv(file_path, dtype='unicode')
   
    # json columns
    json_columns = ['spoken_languages']
    for column in json_columns:
       
        # use ast because json data has single quotes in the csv, which is invalid for a json object; it should be " normally
        df[column] = df[column].apply(lambda x: np.nan if pd.isnull(x) else ast.literal_eval(x))
    return df

In [10]:
movies = load_movies_metadata(r"movies_metadata.csv")
movies_df= movies[["id","original_language","spoken_languages"]].dropna()

movies_df.to_csv('movies_df.csv', index=False)
#movies_df.head()
     

In [11]:

movies_df.loc[:,'language type']="O"
org_lang_df = movies_df[["id","original_language","language type"]]
#org_lang_df.columns = ['Movie Id','Language','Language Type']
org_lang_df.columns = ['movie_id','movie_language','movie_language_type']
org_lang_df.head()

,movie_id,movie_language,movie_language_type
0,862,en,O
1,8844,en,O
2,15602,en,O
3,31357,en,O
4,11862,en,O


In [12]:
movie_id=[]
sp_language=[]
sp_language_type =[]

for i,row in movies_df.iterrows():
    if row['spoken_languages'] is not np.nan:
        a = row['spoken_languages']
        for item in a:
            if item['iso_639_1'] is not np.nan:
                sp_language.append(item['iso_639_1'])
                movie_id.append(row.id)
                sp_language_type.append("S")

language_df= pd.DataFrame({"movie_id":movie_id, "movie_language":sp_language,"movie_language_type":sp_language_type})                
                
language_df.head()
        

,movie_id,movie_language,movie_language_type
0,862,en,S
1,8844,en,S
2,8844,fr,S
3,15602,en,S
4,31357,en,S


In [13]:
movie_languge_df = pd.concat([org_lang_df,language_df], ignore_index =True)
movie_languge_df.head()

,movie_id,movie_language,movie_language_type
0,862,en,O
1,8844,en,O
2,15602,en,O
3,31357,en,O
4,11862,en,O


In [14]:
keywords_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [15]:
movie_id =[]
keyword_id =[]
key_name =[]

for j,row in keywords_df.iterrows():
    key_dict = eval(row['keywords'])
    #if row['keywords'] is not np.nan:
     #   a = row['keywords']
    for i in range(len(key_dict)):
        #if key['name'] is not np.nan:
        #keyword_id.append('id')
        movie_id.append(row.id)
        #key_name.append('name')
        try:
            if key_dict[i]['name'] is not np.nan: 
                keyword_id.append(key_dict[i]['id'])
                key_name.append(key_dict[i]['name'])
        except:
            keyword_id.append("n/a")
            key_name.append("n/a")
            break
key_df = pd.DataFrame({"movie_id":movie_id,"keyword_id":keyword_id,"keyword_name":key_name}).dropna()       
key_df.head()       

,movie_id,keyword_id,keyword_name
0,862,931,jealousy
1,862,4290,toy
2,862,5202,boy
3,862,6054,friendship
4,862,9713,friends


In [ ]:
#loading sql
movies_language = movie_languge_df[["movie_id", "movie_language", "movie_language_type"]].copy()
movies_language.head()

In [ ]:
movie_keywords = pd.DataFrame({"movie_id":movie_id,"keyword_id":keyword_id,"keyword_name":key_name}).dropna()
movie_keywords.head()

In [16]:
keywords_master = pd.DataFrame({"keyword_id":keyword_id,"keyword_name":key_name}).dropna()
keywords_master = keywords_master.drop_duplicates()
keywords_master.head()

,keyword_id,keyword_name
0,931,jealousy
1,4290,toy
2,5202,boy
3,6054,friendship
4,9713,friends


In [17]:
movie_keywords = pd.DataFrame({"movie_id":movie_id,"keyword_id":keyword_id}).dropna()
movie_keywords = movie_keywords.drop_duplicates()
movie_keywords.head()

,movie_id,keyword_id
0,862,931
1,862,4290
2,862,5202
3,862,6054
4,862,9713


In [19]:
rds_connection_string = "root:password@localhost/movies_database"
engine = create_engine(f'mysql://{rds_connection_string}')


In [ ]:
keywords_master.to_sql(name='keywords_master', con=engine, if_exists='append', index=False)


In [20]:
movie_keywords.to_sql(name='movie_keywords', con=engine, if_exists='append', index=False)


In [ ]:
movies_language.to_sql(name='movies_language', con=engine, if_exists='append', index=False)
